In [5]:
import pandas as pd
import numpy as np

import lightgbm as lgb
from catboost import CatBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, make_scorer
from sklearn.linear_model import LogisticRegression
import data_cleaner as dc  # self-writtten python functions
pd.set_option('display.max_rows', 500)  # let pd.df display() 500 rows
pd.set_option('display.max_columns', 500)

# Import data & pre-processing

In [6]:
# with NaN
x_train = pd.read_csv("../clean_data/mci_wv1go.csv")    
x_test = pd.read_csv("../clean_data/mci_wv23.csv")
y_train = x_train.pop("progress")
y_test = x_test.pop("progress")

# imputed:  use 1 data to build pipeline first
x_train_imp = pd.read_csv("../clean_data/mci_wv1go_imp.csv")   
x_train_imp.pop("progress")
x_test_imp = pd.read_csv("../clean_data/mci_wv1go_imp.csv")   
x_test_imp.pop("progress")
dc.overview(x_train_imp)

size: (744, 52)


,dtype,NaN_percentage,unique
SITE,int64,0.0,"[22, 100, 67, 23, 99, 18, 7, 136, 123, 127, 27..."
AGE,float64,0.0,"[67.5, 80.4, 76.8, 72.8, 85.9, 66.5, 81.0, 77...."
EcogPtDivatt,float64,0.0,"[2.5, 1.75, 1.25, 2.25, 4.0, 1.5, 2.0, 3.25, 1..."
EcogPtTotal,float64,0.0,"[1.87179, 1.48718, 1.17949, 3.25641, 1.41026, ..."
EcogSPMem,float64,0.0,"[2.875, 2.25, 2.0, 2.5, 1.125, 3.625, 3.875, 3..."
EcogSPLang,float64,0.0,"[2.55556, 1.11111, 3.66667, 1.44444, 1.33333, ..."
EcogSPVisspat,float64,0.0,"[1.85714, 1.0, 3.42857, 1.83333, 3.0, 3.14286,..."
EcogSPPlan,float64,0.0,"[2.2, 1.4, 3.6, 1.0, 3.2, 2.8, 1.6, 1.5, 2.0, ..."
EcogSPOrgan,float64,0.0,"[1.4, 2.0, 4.0, 1.16667, 2.4, 3.2, 1.66667, 1...."
EcogSPDivatt,float64,0.0,"[2.5, 1.5, 4.0, 2.25, 1.25, 3.33333, 2.75, 1.0..."


In [7]:
# standardization in training & keep the original mean & sd
x_train, train_mean_sd = dc.standardization(x_train)
x_test, test_mean_sd = dc.standardization(x_test)
x_train_imp, train_imp_mean_sd = dc.standardization(x_train_imp)
x_test_imp, test_imp_mean_sd = dc.standardization(x_test_imp)
dc.overview(x_train)

size: (744, 52)


,dtype,NaN_percentage,unique
AV45,float64,0.545699,"[nan, -1.3800090946375136, -0.3280925709345532..."
EcogSPOrgan,float64,0.520161,"[nan, -0.005076892283575777, -0.85868395465098..."
EcogSPVisspat,float64,0.50672,"[nan, -0.7890891236249788, -0.0628226972508486..."
EcogSPDivatt,float64,0.505376,"[nan, -0.4906109963546069, 1.0325458212663021,..."
EcogPtOrgan,float64,0.502688,"[nan, 0.10779612130927581, -0.9480053563253547..."
EcogSPPlan,float64,0.5,"[nan, -0.8465451730664295, -0.2511023852985982..."
EcogPtVisspat,float64,0.498656,"[nan, -0.8051101912992247, -0.0278110367801190..."
EcogSPTotal,float64,0.495968,"[nan, 0.04256446692757493, -0.6203269373807363..."
EcogSPMem,float64,0.495968,"[nan, 0.6728696707207861, -0.7428328768551932,..."
EcogPtDivatt,float64,0.494624,"[nan, -0.9007830511464936, -0.2211286912132780..."


In [ ]:
import xgboost as xgb